In [11]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings('ignore')

plt.style.use('ggplot')

In [24]:
cex_list = ['Bitmax', 'Binance', 'BingX', 'Bitbox', 
 'Bitfinex', 'BitGo', 'Bitkub', 'BitPanda',
 'Bitrue', 'Bitso', 'Bitstamp', 'Bittrex',
 'Bitvavo', 'BitZ', 'BKEX', 'Blockchain.com', 
 'BTC Markets', 'BTCTurk', 'ByBit', 'Changelly',
 'Coinbase', 'Coinex', 'Coinlist', 'CoinSpot',
 'Crypto.com', 'Gate.io', 'Hotbit', 'Indodax',
 'Koibanx', 'Korbit', 'Kraken', 'Kucoin',
 'MEXC', 'Netcoins', 'OkCoin', 'OKX',
 'Paribu', 'Phemex', 'ProBit', 'Ripio', 
 'UpHold', 'Vindax', 'WhiteBIT']

dfs = []

# Loop over each CEX name in the list
for name in cex_list:
    # Construct the file path
    file_path = f'CEX_balances/{name}_balance.csv'
    
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    df.drop(['Entity', 'AlgoBalanceDelta'], axis=1, inplace=True)
    # Rename the 'AlgoBalance' column to include the CEX name, e.g., 'Bitfinex Balance'
    df.rename(columns={'AlgoBalance': f'{name} Balance'}, inplace=True)

    # Append the DataFrame to the list
    dfs.append(df)

# Merge all DataFrames on 'Date', assuming 'Date' is the common column
# We use 'outer' join to make sure we don't lose any data when dates don't match exactly
merged_df = dfs[0]
for df in dfs[1:]:
    merged_df = pd.merge(merged_df, df, on='Date', how='outer')

# Convert 'Date' to datetime format for consistency
merged_df['Date'] = pd.to_datetime(merged_df['Date'])

# Sort the DataFrame by Date
merged_df.sort_values(by='Date', inplace=True)

# Reset the index if needed
merged_df.reset_index(drop=True, inplace=True)



In [28]:
balance_columns = [col for col in merged_df.columns if col.endswith(' Balance')]

# Calculate the total balance for each row by summing across the selected columns
merged_df['Total'] = merged_df[balance_columns].sum(axis=1)
merged_df

,Date,Bitmax Balance,Binance Balance,BingX Balance,Bitbox Balance,Bitfinex Balance,BitGo Balance,Bitkub Balance,BitPanda Balance,Bitrue Balance,...,OkCoin Balance,OKX Balance,Paribu Balance,Phemex Balance,ProBit Balance,Ripio Balance,UpHold Balance,Vindax Balance,WhiteBIT Balance,Total
0,2019-06-11,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000,0.000000,0.000000e+00,0.000000e+00,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.00000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00
1,2019-06-12,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000,0.000000,0.000000e+00,0.000000e+00,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.00000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00
2,2019-06-13,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000,0.000000,0.000000e+00,0.000000e+00,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.00000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00
3,2019-06-14,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000,0.000000,0.000000e+00,0.000000e+00,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.00000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00
4,2019-06-15,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000,0.000000,0.000000e+00,0.000000e+00,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.00000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1944,2024-10-06,178352.232123,5.632900e+08,171494.760786,4645.409785,6.778760e+06,0.198,354435.689525,1.192864e+08,1.013210e+06,...,873234.161633,4.551592e+07,1.016385e+07,1.667939e+06,33018.09019,8072.778710,7.136098e+08,538.566099,467287.373935,2.891409e+09
1945,2024-10-07,177642.748577,5.618988e+08,151888.682254,4645.409785,6.709365e+06,0.198,324715.733735,1.193819e+08,9.797801e+05,...,873003.161901,4.568637e+07,1.014166e+07,1.605693e+06,31445.39049,8072.778710,7.139044e+08,538.566099,467287.373935,2.893311e+09
1946,2024-10-08,175875.645578,5.644460e+08,146730.346527,4645.409785,6.709363e+06,0.198,393750.260185,1.194695e+08,1.038735e+06,...,873003.161901,4.600662e+07,1.014983e+07,1.610541e+06,31445.39049,8072.777710,7.138686e+08,538.566099,467397.262936,2.893970e+09
1947,2024-10-09,177119.427580,5.691440e+08,129048.764697,4645.409785,6.718784e+06,0.198,398252.965185,1.195265e+08,1.019635e+06,...,873003.161901,4.589181e+07,1.017584e+07,1.619799e+06,31445.39049,7921.068726,7.138247e+08,538.566099,471913.777741,2.890031e+09


In [31]:
merged_df.to_csv('data/CEX_balances.csv', index=False)

In [37]:
merged_df['Date'] = pd.to_datetime(merged_df['Date'])

# Step 2: Get the last date in the dataset
last_date = merged_df['Date'].max()
# Step 3: Get the last day balances
last_day_df = merged_df[merged_df['Date'] == last_date]

# Step 4: Prepare ranking DataFrame
ranking_data = {'CEX': [], 'Balance': [], '24h%': [], '7d%': [], '30d%': [], '90d%': []}

# Get the list of CEX columns
cex_columns = [col for col in merged_df.columns if col.endswith(' Balance')]

# Step 5: Populate the ranking data
for cex in cex_columns:
    balance = last_day_df[cex].values[0]  # Get the last day balance
    ranking_data['CEX'].append(cex.replace(' Balance', ''))  # Remove ' Balance' from CEX name
    ranking_data['Balance'].append(balance)
    
    # For 24% Changes, we need the previous day's balance
    previous_day_balance = merged_df[merged_df['Date'] == (last_date - pd.Timedelta(days=1))][cex]
    if not previous_day_balance.empty:
        change_24h = ((balance - previous_day_balance.values[0]) / previous_day_balance.values[0]) * 100
    else:
        change_24h = None
    ranking_data['24h%'].append(change_24h)
    
    # Calculate percentage changes for 7d, 30d, and 90d
    for days in [7, 30, 90]:
        past_date = last_date - pd.Timedelta(days=days)
        past_balance = merged_df[merged_df['Date'] == past_date][cex]
        if not past_balance.empty:
            change = ((balance - past_balance.values[0]) / past_balance.values[0]) * 100
        else:
            change = None
        ranking_data[f'{days}d%'].append(change)

# Step 6: Create the ranking DataFrame
ranking_df = pd.DataFrame(ranking_data)

# Step 7: Sort the ranking DataFrame by Balance in descending order
ranking_df.sort_values(by='Balance', ascending=False, inplace=True)
ranking_df.reset_index(inplace=True, drop=True)

In [39]:
ranking_df.to_csv('data/CEX_ranking.csv')